In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import KFold
pd.set_option('display.max_columns', 50)

In [3]:
train = pd.read_csv('train_indessa.csv')
test = pd.read_csv('test_indessa.csv')

In [4]:
result=train.loan_status.values
train.drop('loan_status', axis = 1, inplace = True)



In [5]:
size_train = train.shape[0]
size_test = test.shape[0]
train = train.append(test)

In [6]:
n = train.isnull().sum(axis=0)
n

member_id                           0
loan_amnt                           0
funded_amnt                         0
funded_amnt_inv                     0
term                                0
batch_enrolled                 130748
int_rate                            0
grade                               0
sub_grade                           0
emp_title                       51462
emp_length                      44825
home_ownership                      0
annual_inc                          4
verification_status                 0
pymnt_plan                          0
desc                           761599
purpose                             0
title                             152
zip_code                            0
addr_state                          0
dti                                 0
delinq_2yrs                        29
inq_last_6mths                     29
mths_since_last_delinq         454312
mths_since_last_record         750326
open_acc                           29
pub_rec     

In [7]:
drop_cols = ['emp_title','mths_since_last_major_derog', 'batch_enrolled', 'member_id', 'mths_since_last_record']
train.drop(drop_cols, axis = 1, inplace = True)

Data Cleaning

In [8]:
def clean_data(raw):
    if ('debt' in raw) or ('consol' in raw) or ('lidation' in raw):
        return 'debt consolidation'
    elif ('credit' in raw) or ('card' in raw):
        return 'credit card'
    elif 'wedd' in raw or 'marr' in raw:
        return 'wedding'
    elif 'home' in raw or 'house' in raw or 'bath' in raw:
        return 'home'
    elif ('car' in raw) or ('motor' in raw) or ('truck' in raw) or ('auto') in raw:
        return 'vehicle'
    elif ('payoff' in raw) or ('off' in raw) :
        return 'payoff'
    elif 'freedom' in raw:
        return 'freedom'
    elif 'pool' in raw:
        return 'pool'
    elif 'persona' in raw :
        return 'personal'
    elif 'vacation' in raw:
        return 'vacation'
    elif 'hosp' in raw or 'med' in raw:
        return 'medical'
    elif 'loan' in raw:
        return 'some loan'
    elif 'movin' in raw:
        return 'moving'
    else :
        return raw
    



In [9]:
train['title'] = train.title.apply(lambda x : x.lower() if type(x) == str else '')
train['title'] = train.title.apply(clean_data)

In [10]:
import re
def get_emp_length(raw):
    if raw == null :
        return np.nan
    elif '+' in raw :
        return 10
    elif '<' in raw :
        return 1
    else :
        return int(re.sub("\D", "", raw))
    



In [40]:
train['emp_length']=train['emp_length'].astype(float)
train.emp_length.dtype

dtype('float64')

In [12]:
#train['emp_length'].astype(float)
#train['emp_length'] = train.emp_length.apply(get_emp_length)
train.emp_length = train.emp_length.astype(str).str.rstrip(' years')
train.emp_length = train.emp_length.astype(str).str.rstrip('+')
train.emp_length = train.emp_length.apply(lambda x : 1 if '<' in x else x)
train['emp_length'].astype(float)

0          9.0
1          1.0
2          2.0
3         10.0
4         10.0
5          2.0
6          5.0
7          8.0
8          7.0
9          2.0
10        10.0
11         4.0
12         7.0
13        10.0
14        10.0
15         4.0
16         9.0
17         NaN
18         7.0
19         7.0
20         9.0
21         2.0
22        10.0
23         7.0
24         1.0
25         4.0
26         2.0
27         1.0
28         3.0
29         2.0
          ... 
354921     1.0
354922     1.0
354923     NaN
354924    10.0
354925     4.0
354926    10.0
354927     6.0
354928    10.0
354929     7.0
354930     4.0
354931     2.0
354932     3.0
354933     4.0
354934     3.0
354935     3.0
354936     5.0
354937    10.0
354938     9.0
354939     3.0
354940     1.0
354941     2.0
354942     9.0
354943     5.0
354944    10.0
354945    10.0
354946     1.0
354947     1.0
354948    10.0
354949     1.0
354950     2.0
Name: emp_length, Length: 887379, dtype: float64

In [13]:
train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,14350,14350,14350.0,36 months,19.19,E,E3,9,OWN,28700.0,Source Verified,n,NaN,debt_consolidation,debt consolidation,349xx,FL,33.88,0.0,1.0,50.0,14.0,1.0,22515.0,73.1,28.0,f,1173.84,0.0,0.0,0.0,0.0,INDIVIDUAL,NaN,26th week,0.0,0.0,28699.0,30800.0
1,4800,4800,4800.0,36 months,10.99,B,B4,1,MORTGAGE,65000.0,Source Verified,n,NaN,home_improvement,home,209xx,MD,3.64,0.0,1.0,NaN,6.0,0.0,7624.0,23.2,13.0,w,83.95,0.0,0.0,0.0,0.0,INDIVIDUAL,NaN,9th week,0.0,0.0,9974.0,32900.0
2,10000,10000,10000.0,36 months,7.26,A,A4,2,OWN,45000.0,Not Verified,n,NaN,debt_consolidation,debt consolidation,447xx,OH,18.42,0.0,0.0,NaN,5.0,0.0,10877.0,31.2,19.0,w,56.47,0.0,0.0,0.0,0.0,INDIVIDUAL,NaN,9th week,0.0,65.0,38295.0,34900.0
3,15000,15000,15000.0,36 months,19.72,D,D5,10,RENT,105000.0,Not Verified,n,> My goal is to obtain a loan to pay off my hi...,debt_consolidation,debt consolidation,221xx,VA,14.97,0.0,2.0,46.0,10.0,0.0,13712.0,55.5,21.0,f,4858.62,0.0,0.0,0.0,0.0,INDIVIDUAL,NaN,135th week,0.0,0.0,55564.0,24700.0
4,16000,16000,16000.0,36 months,10.64,B,B2,10,RENT,52000.0,Verified,n,NaN,credit_card,refi,900xx,CA,20.16,0.0,0.0,NaN,11.0,0.0,35835.0,76.2,27.0,w,2296.41,0.0,0.0,0.0,0.0,INDIVIDUAL,NaN,96th week,0.0,0.0,47159.0,47033.0


In [14]:
train=train.drop('verification_status_joint',1)
train=train.drop('addr_state',1)


In [15]:
sorted(train)

['acc_now_delinq',
 'annual_inc',
 'application_type',
 'collection_recovery_fee',
 'collections_12_mths_ex_med',
 'delinq_2yrs',
 'desc',
 'dti',
 'emp_length',
 'funded_amnt',
 'funded_amnt_inv',
 'grade',
 'home_ownership',
 'initial_list_status',
 'inq_last_6mths',
 'int_rate',
 'last_week_pay',
 'loan_amnt',
 'mths_since_last_delinq',
 'open_acc',
 'pub_rec',
 'purpose',
 'pymnt_plan',
 'recoveries',
 'revol_bal',
 'revol_util',
 'sub_grade',
 'term',
 'title',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_acc',
 'total_rec_int',
 'total_rec_late_fee',
 'total_rev_hi_lim',
 'verification_status',
 'zip_code']

In [41]:
def get_last_week_pay(raw) :
    try :
        return int(re.sub("\D", "", raw))
    except :
        return -9999
    
train['last_week_pay'] = train['last_week_pay'].apply(get_last_week_pay)
train.term = train.term.astype(str).str.rstrip(' months')
train['term']=train['term'].astype(float)
train.zip_code = train.zip_code.astype(str).str.rstrip('xxx')
train['zip_code']=train['zip_code'].astype(float)

               

In [17]:
train=train.drop('desc',1)

Adding new features to compute


In [70]:
train['Interest'] = train['funded_amnt']*train['int_rate']*train['term']/(100*12)
train['Monthly_income'] = train['annual_inc'].apply(lambda x : x/12 if x >=0 else -9999)
train['unpaid_perc'] = (train['Interest'] - train['total_rec_int'])/train['Interest']

C:\ProgramData\Anaconda4\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda4\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda4\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [67]:
cols = []
for col in train.columns:
    if col not in ['Interest', 'total_rec_int']:
        cols.append(col)

TO get list of object column to apply encoder

In [42]:
cols_list = train.columns
cols_obj = []
obj_list = train.dtypes.values
for col, obj in zip(cols_list, obj_list):
    if obj == object:
        cols_obj.append(col)
        
train[cols_obj].count()

home_ownership         407770
verification_status    407770
pymnt_plan             407770
purpose                407770
title                  407770
initial_list_status    407770
application_type       407770
dtype: int64

In [43]:
cols_obj

['home_ownership',
 'verification_status',
 'pymnt_plan',
 'purpose',
 'title',
 'initial_list_status',
 'application_type']

In [45]:
le = preprocessing.LabelEncoder()
for col in cols_obj:
    train[col] = le.fit_transform(train[col])

In [68]:
train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,application_type,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,Interest,Monthly_income,unpaid_perc
0,14350,14350,14350.0,0.0,19.19,4,22,9.0,4,28700.0,1,0,2,1295,349.0,33.88,0.0,1.0,50.0,14.0,1.0,22515.0,73.1,28.0,0,1173.84,0.0,0.0,0.0,0.0,0,-9999,0.0,0.0,28699.0,30800.0,0.0,2391.666667,-inf
3,15000,15000,15000.0,0.0,19.72,3,19,10.0,5,105000.0,0,0,2,1295,221.0,14.97,0.0,2.0,46.0,10.0,0.0,13712.0,55.5,21.0,0,4858.62,0.0,0.0,0.0,0.0,0,-9999,0.0,0.0,55564.0,24700.0,0.0,8750.000000,-inf
5,15000,15000,14950.0,0.0,8.90,0,4,2.0,1,120000.0,0,0,2,1295,461.0,12.30,0.0,0.0,56.0,18.0,0.0,19040.0,64.5,30.0,0,1957.24,0.0,0.0,0.0,0.0,0,-9999,0.0,0.0,350619.0,29500.0,0.0,10000.000000,-inf
7,6000,6000,6000.0,0.0,9.17,1,5,8.0,1,54000.0,0,0,1,1218,351.0,11.63,0.0,1.0,46.0,13.0,0.0,3484.0,29.5,49.0,0,637.51,0.0,0.0,0.0,0.0,0,-9999,0.0,0.0,272579.0,11800.0,0.0,4500.000000,-inf
8,6000,6000,6000.0,0.0,13.99,2,13,7.0,1,92000.0,0,0,4,2493,917.0,30.85,0.0,0.0,77.0,16.0,0.0,47567.0,76.6,27.0,1,621.72,0.0,0.0,0.0,0.0,0,-9999,0.0,0.0,281521.0,62100.0,0.0,7666.666667,-inf


In [55]:
train_new = train[:size_train]
test_new = train[size_train:]

def ceate_feature_map(features):
    outfile = open('xgb_new.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

def xgb_feature_importance(features, model):
    ceate_feature_map(features)
    
    import operator
    importance = model.get_fscore(fmap='xgb_new.fmap')
    importance = sorted(importance.items(), key=operator.itemgetter(1))
    
    df = pd.DataFrame(importance, columns=['feature', 'fscore'])
    df['fscore'] = df['fscore'] / df['fscore'].sum()
    plt.figure()
    df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
    plt.title('XGBoost Feature Importance')
    plt.xlabel('relative importance')
    plt.show()
    
    return df
    
featuredf = xgb_feature_importance(test_new.columns, model)


In [56]:
import xgboost as xgb
param = {}
param['objective'] = 'binary:logistic'
param['eta'] = 0.1
param['max_depth'] = 10
param['silent'] = 0
param['eval_metric'] = "auc"
param['subsample'] = 1
param['colsample_bytree'] = 1
param['seed'] = 3
num_rounds = 5000

In [61]:
def train_xgb(X_train, y_train, X_val, y_val):
    
    dtrain = xgb.DMatrix(X_train, label=y_train, missing=-9999)
    
    dtest = xgb.DMatrix(X_val, label=y_val, missing=-9999)
    
    watchlist  = [(dtest,'test'), (dtrain,'train')]
    bst = xgb.train(param, dtrain, num_rounds, watchlist, early_stopping_rounds=20)
    
    return bst
    #print(get_log_loss(y_val, pred), get_log_loss(y_train, predtrain))

In [62]:
kf = KFold(n_splits=2, shuffle=True, random_state=3)

for train_index, test_index in kf.split(train_new):    
    print("Split")
    model = train_xgb(train_new.iloc[train_index].values, result[train_index], train_new.iloc[test_index].values, result[test_index])

Split
[0]	test-auc:0.502194	train-auc:0.531354
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 2 rounds.
[1]	test-auc:0.500782	train-auc:0.542678
[2]	test-auc:0.500989	train-auc:0.557447
[3]	test-auc:0.500436	train-auc:0.568098
[4]	test-auc:0.499613	train-auc:0.57923
[5]	test-auc:0.499814	train-auc:0.586638
[6]	test-auc:0.500592	train-auc:0.591792
[7]	test-auc:0.500289	train-auc:0.602853
[8]	test-auc:0.500425	train-auc:0.60734
[9]	test-auc:0.500447	train-auc:0.611957
[10]	test-auc:0.499928	train-auc:0.616329
[11]	test-auc:0.499949	train-auc:0.620382
[12]	test-auc:0.499962	train-auc:0.62609
[13]	test-auc:0.49977	train-auc:0.626767
[14]	test-auc:0.499418	train-auc:0.632191
[15]	test-auc:0.499495	train-auc:0.632315
[16]	test-auc:0.499949	train-auc:0.633297
[17]	test-auc:0.500285	train-auc:0.647759
[18]	test-auc:0.499872	train-auc:0.648226
[19]	test-auc:0.500153	train-auc:0.649255
[20]	test-auc:0.500231	tra

[191]	test-auc:0.498689	train-auc:0.906544
[192]	test-auc:0.498695	train-auc:0.906553
[193]	test-auc:0.498655	train-auc:0.907172
[194]	test-auc:0.498598	train-auc:0.908166
[195]	test-auc:0.498562	train-auc:0.909371
[196]	test-auc:0.498616	train-auc:0.909421
[197]	test-auc:0.49876	train-auc:0.90965
[198]	test-auc:0.498617	train-auc:0.911057
[199]	test-auc:0.498626	train-auc:0.912999
[200]	test-auc:0.498704	train-auc:0.913706
[201]	test-auc:0.498521	train-auc:0.914642
[202]	test-auc:0.498594	train-auc:0.915144
[203]	test-auc:0.498602	train-auc:0.915169
[204]	test-auc:0.498295	train-auc:0.916277
[205]	test-auc:0.49852	train-auc:0.916881
[206]	test-auc:0.498528	train-auc:0.916894
[207]	test-auc:0.498651	train-auc:0.918208
[208]	test-auc:0.498494	train-auc:0.91894
[209]	test-auc:0.498481	train-auc:0.919001
[210]	test-auc:0.498593	train-auc:0.919562
[211]	test-auc:0.498585	train-auc:0.919577
[212]	test-auc:0.498693	train-auc:0.920098
[213]	test-auc:0.498635	train-auc:0.920135
[214]	test-auc:

[383]	test-auc:0.498987	train-auc:0.976641
[384]	test-auc:0.498821	train-auc:0.977066
[385]	test-auc:0.498829	train-auc:0.977304
[386]	test-auc:0.498852	train-auc:0.977461
[387]	test-auc:0.498893	train-auc:0.977571
[388]	test-auc:0.498903	train-auc:0.977585
[389]	test-auc:0.498853	train-auc:0.9777
[390]	test-auc:0.498744	train-auc:0.978053
[391]	test-auc:0.49877	train-auc:0.97829
[392]	test-auc:0.498772	train-auc:0.978291
[393]	test-auc:0.498685	train-auc:0.97832
[394]	test-auc:0.498685	train-auc:0.978611
[395]	test-auc:0.498646	train-auc:0.97866
[396]	test-auc:0.498603	train-auc:0.978774
[397]	test-auc:0.498503	train-auc:0.978839
[398]	test-auc:0.49858	train-auc:0.978911
[399]	test-auc:0.498598	train-auc:0.979062
[400]	test-auc:0.498607	train-auc:0.979063
[401]	test-auc:0.498561	train-auc:0.979145
[402]	test-auc:0.498554	train-auc:0.979387
[403]	test-auc:0.498516	train-auc:0.979581
[404]	test-auc:0.498536	train-auc:0.979852
[405]	test-auc:0.498512	train-auc:0.979933
[406]	test-auc:0.4

[575]	test-auc:0.498602	train-auc:0.991911
[576]	test-auc:0.498608	train-auc:0.991912
[577]	test-auc:0.49862	train-auc:0.991913
[578]	test-auc:0.498648	train-auc:0.991914
[579]	test-auc:0.498647	train-auc:0.991914
[580]	test-auc:0.49862	train-auc:0.991933
[581]	test-auc:0.498618	train-auc:0.991973
[582]	test-auc:0.498635	train-auc:0.991979
[583]	test-auc:0.498656	train-auc:0.991981
[584]	test-auc:0.498644	train-auc:0.992028
[585]	test-auc:0.498563	train-auc:0.992151
[586]	test-auc:0.498562	train-auc:0.992156
[587]	test-auc:0.498582	train-auc:0.992163
[588]	test-auc:0.498606	train-auc:0.992168
[589]	test-auc:0.498632	train-auc:0.992178
[590]	test-auc:0.498601	train-auc:0.992277
[591]	test-auc:0.498619	train-auc:0.992338
[592]	test-auc:0.498645	train-auc:0.992355
[593]	test-auc:0.498684	train-auc:0.992396
[594]	test-auc:0.498738	train-auc:0.992493
[595]	test-auc:0.498714	train-auc:0.992575
[596]	test-auc:0.498672	train-auc:0.992656
[597]	test-auc:0.498591	train-auc:0.992788
[598]	test-au

KeyboardInterrupt: 

In [ ]:
def ceate_feature_map(features):
    outfile = open('xgb_new.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

def xgb_feature_importance(features, model):
    ceate_feature_map(features)
    
    import operator
    importance = model.get_fscore(fmap='xgb_new.fmap')
    importance = sorted(importance.items(), key=operator.itemgetter(1))
    
    df = pd.DataFrame(importance, columns=['feature', 'fscore'])
    df['fscore'] = df['fscore'] / df['fscore'].sum()
    plt.figure()
    df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
    plt.title('XGBoost Feature Importance')
    plt.xlabel('relative importance')
    plt.show()
    
    return df

featuredf = xgb_feature_importance(test_new.columns, model)

In [63]:
def train_and_predict(X_train, y_train, X_val, y_val, X_test):
    dtrain = xgb.DMatrix(X_train, label=y_train, missing=-9999)
    dtest = xgb.DMatrix(X_val, label=y_val, missing=-9999)
    watchlist  = [(dtest,'test'), (dtrain,'train')]
    bst = xgb.train(param, dtrain, num_rounds, watchlist, early_stopping_rounds=20)
    
    bst.save_model('mymodel')
    bst = xgb.Booster(param)
    bst.load_model('mymodel')
    
    #xgboost issue : https://github.com/dmlc/xgboost/issues/1238
    
    dpredict = xgb.DMatrix(X_test, missing=-9999)
    pred = bst.predict(dpredict)
    return pred

In [52]:
train.isnull().sum(axis=0)
train=train.dropna()

In [64]:
import xgboost as xgb
param = {}
param['objective'] = 'binary:logistic'
param['eta'] = 0.03
param['max_depth'] = 10
param['silent'] = 0
param['eval_metric'] = "auc"
param['subsample'] = 1
param['colsample_bytree'] = 1
param['seed'] = 3
num_rounds = 5000

In [65]:
### Final predict
kf = KFold(n_splits=5, shuffle=True, random_state=3)
pred_arr = []

for train_index, test_index in kf.split(train_new):    
    print("Split")
    pred_temp = train_and_predict(train_new.iloc[train_index].values, result[train_index], train_new.iloc[test_index].values, result[test_index], test_new)
    print(pred_temp[1:15])
    pred_arr.append(pred_temp)

Split
[0]	test-auc:0.499945	train-auc:0.516086
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 20 rounds.
[1]	test-auc:0.501523	train-auc:0.518866
[2]	test-auc:0.501042	train-auc:0.520458
[3]	test-auc:0.502313	train-auc:0.527246


KeyboardInterrupt: 

In [ ]:
pred1 = pred_arr[0]
pred2 = pred_arr[1]
pred3 = pred_arr[2]
pred4 = pred_arr[3]
pred5 = pred_arr[4]

In [ ]:
pred_avg = (pred1+pred2+pred3+pred4+pred5)/5

In [ ]:
pred_avg = (pred1+pred2+pred3+pred4+pred5)/5
print(pred_avg[1:5])

In [ ]:
test = pd.read_csv('test_indessa.csv')
member_id = test.member_id

In [ ]:
ans = pd.DataFrame({
    'member_id' : member_id,
    'loan_status' : pred_avg
})
ans = ans[['member_id','loan_status']]

In [ ]:
ans.to_csv('answer_f_5000_rounds_eta_03_1.csv', index = False) ### 0.97892 on lb and (0.97927 after ensemble with prev models)